In [6]:

import csv
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException



# Configuration constants
URL = "https://www.randhawa.us/games/retailer/nyu.html"
TARGET_TOTAL = 2000  # Total inventory
WEEKS = 15           # Game duration in weeks
SIMULATIONS = 2   # Number of simulation rounds to run
WAIT_TIMEOUT = 2    # Maximum seconds to wait for elements

# Button IDs/text for locating elements (adjust as needed based on actual page)
BUTTONS = {
    'restart': "practiceButton",       # Restart Game
    'maintain': "maintainButton",      # Maintain Price
    'md10': "tenButton",               # 10% Markdown
    'md20': "twentyButton",            # 20% Markdown
    'md40': "fortyButton"              # 40% Markdown
}


In [7]:




def creeping_strategy():
    """Strategy generator that decides markdown actions based on sales pace.
    
    This strategy compares cumulative sales to an ideal linear sales pace and
    applies larger markdowns when falling behind target pace.
    
    Yields:
        str: Action key for the next decision ('maintain', 'md10', 'md20', or 'md40')
    """
    week = 1
    cum_sales = 0
    
    # Thresholds for markdown decisions (as percentage of ideal pace)
    thresholds = {
        'md10': 0.75,  # below 75% of ideal pace → 10% markdown
        'md20': 0.50,  # below 50% → 20% markdown
        'md40': 0.25,  # below 25% → 40% markdown
    }
    
    # Initial action for week 1
    action = 'maintain'
    cum_sales = yield action
    
    while week < WEEKS:
        # Calculate sales pace as ratio of actual to ideal
        ideal_sales = (week / WEEKS) * TARGET_TOTAL
        pace_ratio = cum_sales / ideal_sales if ideal_sales > 0 else 0
        
        # Final week strategy - maximize sales with deep markdown
        if week >= WEEKS - 1:
            action = 'md40'
        # Choose markdown based on how far behind pace we are
        elif pace_ratio < thresholds['md40']:
            action = 'md40'
        elif pace_ratio < thresholds['md20']:
            action = 'md20'
        elif pace_ratio < thresholds['md10']:
            action = 'md10'
        else:
            action = 'maintain'
        
        # Yield decision and receive updated sales data
        cum_sales = yield action
        week += 1

def play_simulation(driver, sim_num, week_writer, strategy_callback):
    """Play a single simulation round of the Retailer Game.
    
    Args:
        driver: Selenium WebDriver instance
        sim_num: Current simulation number (for logging)
        week_writer: CSV writer for weekly results
        strategy_callback: Generator providing action decisions
        
    Returns:
        float: Total revenue earned in this simulation
    """
    # Wait for and click the restart button
    try:
        restart_button = WebDriverWait(driver, WAIT_TIMEOUT).until(
            EC.element_to_be_clickable((By.ID, BUTTONS['restart']))
        )
        restart_button.click()
    except TimeoutException:
        print(f"Simulation {sim_num}: Timeout waiting for restart button")
        return 0.0
    
    # Initialize tracking variables
    week = 1
    inventory = TARGET_TOTAL
    total_revenue = 0.0
    
    # Track which markdown levels have been used (to enforce one-way progression)
    used_markdowns = {
        'md10': False,
        'md20': False,
        'md40': False
    }
    
    # Continue until game end conditions
    while week <= WEEKS:
        try:
            # Wait for table to update with current week's results
            table = WebDriverWait(driver, WAIT_TIMEOUT).until(
                EC.presence_of_element_located((By.TAG_NAME, "tbody"))
            )
            
            # Get the last (most recent) row
            rows = table.find_elements(By.TAG_NAME, "tr")
            if not rows:
                print(f"Simulation {sim_num}: No data rows found")
                break
                
            last_row = rows[-1]
            cells = last_row.find_elements(By.TAG_NAME, "td")
            
            if len(cells) < 4:
                print(f"Simulation {sim_num}: Incomplete data row")
                break
            
            # Parse values from table cells
            try:
                current_week = int(cells[0].text.strip())
                price = float(cells[1].text.strip().replace('$', ''))
                sales = int(cells[2].text.strip())
                inventory = int(cells[3].text.strip())
            except (ValueError, IndexError) as e:
                print(f"Simulation {sim_num}: Error parsing table data: {e}")
                break
            
            # Update total revenue and write data
            total_revenue += price * sales
            week_writer.writerow([sim_num, current_week, price, sales, inventory])
            
            # Check for game end conditions
            if current_week == WEEKS or inventory == 0:
                break
                
            # Determine which actions are allowed based on markdown progression
            allowed_actions = {'maintain': BUTTONS['maintain']}
            if not used_markdowns['md40']:
                allowed_actions['md40'] = BUTTONS['md40']
            if not used_markdowns['md20'] and not used_markdowns['md40']:
                allowed_actions['md20'] = BUTTONS['md20']
            if not used_markdowns['md10'] and not used_markdowns['md20'] and not used_markdowns['md40']:
                allowed_actions['md10'] = BUTTONS['md10']
            
            # Get strategy decision
            cumulative_sales = TARGET_TOTAL - inventory
            action_key = strategy_callback.send(cumulative_sales)
            
            # Validate the action is allowed
            if action_key not in allowed_actions:
                raise ValueError(f"Invalid action '{action_key}' for week {current_week}")
            
            # Click the selected button
            button = driver.find_element(By.ID, allowed_actions[action_key])
            button.click()
            
            # Update the markdown usage status
            if action_key == 'md10':
                used_markdowns['md10'] = True
            elif action_key == 'md20':
                used_markdowns['md10'] = True  # Implicit usage
                used_markdowns['md20'] = True
            elif action_key == 'md40':
                used_markdowns['md10'] = True  # Implicit usage
                used_markdowns['md20'] = True  # Implicit usage
                used_markdowns['md40'] = True
                
            week += 1
            
        except (TimeoutException, NoSuchElementException) as e:
            print(f"Simulation {sim_num}: Error in week {week}: {e}")
            break
            
    return total_revenue


In [9]:

def main():
    """Main function to run all simulations and save results."""
    # Configure Chrome in headless mode
    options = webdriver.ChromeOptions()
    #options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    
    # Create webdriver
    driver = webdriver.Chrome(options=options)
    driver.set_window_size(1366, 768)  # Set window size for consistent rendering
    
    try:
        # Open CSV files for writing results
        with open('week_detail.csv', 'w', newline='') as detail_file, \
             open('outcome.csv', 'w', newline='') as outcome_file:
            
            # Create CSV writers and write headers
            week_writer = csv.writer(detail_file)
            outcome_writer = csv.writer(outcome_file)
            
            week_writer.writerow(['Simulation Number', 'Week', 'Price', 'Sales', 'Remaining Inventory'])
            outcome_writer.writerow(['Simulation Number', 'Your revenue', 'Perfect foresight strategy', 'Difference (%)'])
            
            # Run all simulations
            for sim_num in range(1, SIMULATIONS + 1):
                print(f"Running simulation {sim_num}/{SIMULATIONS}")
                
                # Initialize the strategy generator
                strategy_callback = creeping_strategy()
                next(strategy_callback)  # Prime the generator
                
                # Navigate to the game URL
                driver.get(URL)
                
                # Play the simulation and get revenue
                revenue = play_simulation(driver, sim_num, week_writer, strategy_callback)
                
                # Calculate placeholder values for perfect strategy and difference
                # In a real implementation, you might have a benchmarking algorithm here
                perfect = revenue * 1.15  # Placeholder - assuming perfect is 15% better
                diff_pct = ((perfect - revenue) / perfect) * 100 if perfect > 0 else 0
                
                # Write outcome results
                outcome_writer.writerow([sim_num, round(revenue, 2), round(perfect, 2), round(diff_pct, 2)])
                
                # Ensure detail file is flushed for monitoring progress
                detail_file.flush()
                
                # Optional: Add a small delay between simulations to prevent overloading
                time.sleep(0.5)
                
    except Exception as e:
        print(f"Error in main execution: {e}")
    finally:
        # Clean up
        driver.quit()
        print("Simulations complete.")

if __name__ == '__main__':
    main()

Running simulation 1/2
Simulation 1: Incomplete data row
Running simulation 2/2
Simulation 2: Incomplete data row
Simulations complete.
